In [ ]:
import numpy as np

# 1 Lineare Algebra: Skalarprodukte

# 2 Lineare Algebra: Matrixmultiplikation

**Definition.** Für zwei Matrizen $M \in \mathbb{R}^{m \times l}$ (d.h. $m$ Zeilen und $l$ Spalten, shape $(m,l)$) und $N \in \mathbb{R}^{l \times n}$ (d.h. $l$ Zeilen und $n$ Spalten, shape $(l,n)$) ist das Matrixprodukt definiert als
$$MN \in \mathbb{R}^{m \times n}, \quad (MN)_{ij} := M_i \cdot (N^\top)_j.$$

Der Eintrag von $MN$ in Zeile $i$ und Spalte $j$ ist also das Skalarprodukt der $i$-ten Zeile von $M$ mit der $j$-ten Spalte von $N$ (das ist gleich der $j$-ten Zeile von $N^\top$).

In Numpy führt `np.matmul` bzw. der `@`-Operator die Matrixmultiplikation aus (nicht `*`).
Man kann natürlich auch direkt schreiben:
$$ (MN)_{ij} = \sum_{k=1}^{l} M_{ik} N_{kj}. $$

## Aufgaben zu Lineare Algebra: Matrixmultiplikation

**Aufgabe 1.**  
Wir können eine ‘lazy’ Matrixmultiplikation implementieren (und dabei Numpy’s `np.dot` nutzen): eine Datenstruktur, die zwei Matrizen $M, N$ abspeichert und erst dann einen Eintrag $(MN)_{ij}$ berechnet, wenn dieser angefordert wird, etwa so:
```python
x = LazyMatMul(M,N)
print(x.getEntry(2,3)) # Hier wird erst gerechnet
```

- **Bonusaufgabe a:** Überschreiben Sie geeignete Python-Methoden, um die Syntax `x[2,3]` oder `x[2][3]` zu ermöglichen.
- **Bonusaufgabe b:** Implementieren Sie Caching, sodass ein zweiter Abruf des selben Eintrags keine erneute Rechnung auslöst.
- **Bonusaufgabe c:** Testen Sie Ihre Implementierung randomisiert gegen Numpy’s eigene Matrixmultiplikation.

In [38]:
class LazyMatMul:
    def __init__(self, M, N):
        self.M, self.N = np.array(M), np.array(N)
        self.cache = np.full((self.M.shape[0], self.N.shape[1]), None, dtype=np.float64)
        assert self.M.shape[1] == self.N.shape[0]

    def getEntry(self, i, j):
        if not np.isnan(self.cache[i, j]): return self.cache[i, j]
        self.cache[i, j] = self.M[i] @ self.N[:, j]
        return self.cache[i, j]

    def __getitem__(self, i):
        if isinstance(i, tuple): return self.getEntry(i[0], i[1])
        return [self.getEntry(i, j) for j in range(self.N.shape[1])]


x = LazyMatMul([[1, 2, 3], [4, 5, 6]], [[1], [2], [3]])
assert x.getEntry(1, 0) == 32
assert x[1, 0] == 32
assert x[1][0] == 32

**Aufgabe 2.**  
Wenn man symmetrische Matrizen multipliziert, also Matrizen $M \in \mathbb{R}^{m \times m}$ mit $M^\top = M$, benötigt man weniger Speicher für die Einträge der Matrix. Das gleiche gilt, wenn man obere Dreiecksmatrizen multiplizieren möchte, also $M \in \mathbb{R}^{m \times m}$ mit $M_{ij} = 0$ für $j > i$. Implementieren Sie eine Datenstruktur `TriangMat` für obere Dreiecksmatrizen, die nur so viele Floats speichert, wie wirklich notwendig sind.  
Als Konstruktor bietet sich an, eine ’klassische’ Matrix (als Numpy Array) anzunehmen und eine Fehlermeldung zu generieren, wenn es sich nicht um eine obere Dreiecksmatrix handelt. Schreiben Sie eine `triangMatMul`-Methode, die zwei obere Dreiecksmatrizen multipliziert.

- **Bonusaufgabe:** Implementieren Sie analog `SymmMat` für symmetrische Matrizen.

In [68]:
class TriangMat:
    def __init__(self, M):
        M = np.array(M)
        assert M.shape[0] == M.shape[1]
        self.M = {i: row[i:] for i, row in enumerate(M)}

    def set(self, i, j, v):
        self.M[i][j - i] = v


x = TriangMat([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
assert np.array_equal(x.M[0], [1, 2, 3])
assert np.array_equal(x.M[1], [5, 6])
assert np.array_equal(x.M[2], [9])

In [90]:
def triangMatMul(M, N):
    O = TriangMat(M)
    for i in range(M.shape[0]):
        for j in range(i, N.shape[1]):
            sum = 0
            for k in range(j + 1):
                sum += M[i, k] * N[k, j]
            O.set(i, j, sum)
    return O

In [91]:
M = np.array([[1, 2, 3], [0, 5, 6], [0, 0, 9]])
triangMatMul(M, M).M

{0: array([ 1, 12, 42]), 1: array([25, 84]), 2: array([81])}

**Aufgabe 3.**  
Gegeben eine Matrix $M \in \mathbb{R}^{m \times l}$ und Zerlegungen $m = m_1 + m_2$ sowie $l = l_1 + l_2$ mit $m_1, m_2, l_1, l_2 > 0$ können wir $M$ als ’Blockmatrix’ schreiben:
$
M =
\begin{pmatrix}
A & B \\
C & D
\end{pmatrix}, A \in \mathbb{R}^{m_1 \times l_1}, B \in \mathbb{R}^{m_1 \times l_2}, C \in \mathbb{R}^{m_2 \times l_1}, D \in \mathbb{R}^{m_2 \times l_2}.
$
Wenn wir ebenso $N \in \mathbb{R}^{l \times n}$ zerlegen (mit $n = n_1 + n_2$ für $n_1, n_2 > 0$), also:
$
N =
\begin{pmatrix}
E & F \\
G & H
\end{pmatrix}, E \in \mathbb{R}^{l_1 \times n_1}, F \in \mathbb{R}^{l_1 \times n_2}, G \in \mathbb{R}^{l_2 \times n_1}, H \in \mathbb{R}^{l_2 \times n_2},
$
dann können wir das Matrixprodukt blockweise ausrechnen:
$
MN =
\begin{pmatrix}
AE + BG & AF + BH \\
CE + DG & CF + DH
\end{pmatrix}.
$

Um $MN$ auszurechnen, müssen $M$ und $N$ also nie gleichzeitig vollständig im Speicher liegen.  
Überlegen Sie: Wie lässt sich das verallgemeinern?

???

# 3 Lineare Algebra: Matrixmultiplikation von dünn besetzten Matrizen

**Definition.**  
Eine Matrix heißt dünn besetzt (sparse), wenn die meisten Einträge 0 sind.

Das ist keine rigorose Definition! Ein häufiges Kriterium ist, dass die Anzahl der nicht-0-Einträge ungefähr der Anzahl Zeilen oder Spalten entsprechen sollte, um von Sparsity zu sprechen.

Tatsächlich liegt die Entscheidung bei uns, ob wir eine Matrix als dünn besetzt auffassen wollen, oder nicht. Der Unterschied liegt darin, wie wir die Einträge der Matrix abspeichern.

Eine 'volle' Matrix erfordert das Speichern aller Einträge. Eine dünn besetzte Matrix erfordert nur das Speichern der nicht-0-Einträge.  
In Python speichern wir volle Matrizen als Listen von Listen oder (besser!) als Numpy Arrays. Dünn besetzte Matrizen können wir z.B. als Dictionaries (= Hashmap) abspeichern:

```python
m_sparse = {(0,1) : 5, (1,1) : 4}
```

Das könnte z.B. die Matrix $M = \begin{pmatrix} 0 & 5 \\ 0 & 4 \end{pmatrix}$ sein. Tatsächlich könnte $M$ auch eine $3 \times 2$-Matrix oder eine $10 \times 10$-Matrix sein (mit vielen Nullen). Manchmal möchte man daher Zeilen- und Spaltenanzahl auch speichern.

**Aufgabe 4:** Implementieren Sie Matrixmultiplikation von zwei dünn besetzten Matrizen

**Tipp:**  
Implementieren Sie zunächst das Berechnen eines einzelnen Eintrags der Produktmatrix.  
Überlegen Sie sich als erstes, wie es geht, wenn die beiden Matrizen ein Zeilenvektor und ein Spaltenvektor sind (sodass das Matrixprodukt eine $1 \times 1$-Matrix ist, die genau das Skalarprodukt als einzigen Eintrag enthält).

In [112]:
def sparse_single(M, N, o, p):
    """o, p are the indices of the output matrix we are computing"""
    return sum(M[i, j] * N[(j, p)]
               for i, j in M
               if i == o
               and (j, p) in N)


def sparse_mat_mul(M, N, m, l):
    """
    M is a sparse matrix of shape [m, _]
    N is a sparse matrix of shape [_, l]
    """
    O = dict()
    for i in range(m):
        for j in range(l):
            res = sparse_single(M, N, i, j)
            if res != 0: O[(i, j)] = res

    return O

In [122]:
M = {(0, 1): 5, (1, 1): 4, (1, 2): 3, (2, 4): 2, (2, 3): 1}
M = sparse_mat_mul(M, M, 5, 5)
M = sparse_mat_mul(M, M, 5, 5)
M

{(0, 1): 320,
 (0, 2): 240,
 (0, 3): 60,
 (0, 4): 120,
 (1, 1): 256,
 (1, 2): 192,
 (1, 3): 48,
 (1, 4): 96}

# 4 Simples Map-Reduce-Framework (octo.py)

**Aufgabe 5:**

Schauen Sie sich den Code octo.py an.  
Was können Sie grob über die Struktur sagen?  
Wie verwendet man das Modul, und was passiert dann?